In [20]:
# Import packages
import numpy as np
import ipdb
import pandas as pd
import sys
eps = np.finfo(float).eps
from numpy import log2 as log

In [21]:
# Load training data, split into train and validation sets
data = pd.read_csv("train.csv")
train_data, val_data = np.split(data, [int(.8*len(data))])
# train_data = data.sample(frac=0.8)
# val_data = data.drop(train_data.index)
discrete_attributes = ["Work_accident", "promotion_last_5years", "sales", "salary"]    
real_attributes = ["satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", "time_spend_company"]

In [22]:
def entropy_before_split(t_data):
    """ Finds Entropy of dataset before any split """
    
    dependent_variable = "left"
    count_0 = len(t_data[t_data[dependent_variable] == 0])
    total = len(t_data)
    ratio = count_0 / total
    gini = min(ratio, (1 - ratio))
    return gini

In [23]:
def entropy_on_real_attribute_split(t_data,attribute):
    Class = "left"   
    target_variables = t_data[Class].unique() 
    variables = t_data[attribute].unique()    
    variables.sort()
    
    split_node_entropy =  sys.float_info.max
    split_node = None
    
    if (len(variables) == 1):
        return split_node_entropy, variables[0]
    
    total_rows  = len(t_data)
    for variable in variables[:-1]:
        entropy = 0
        entropy2 = 0
        
        num = len(t_data[t_data[attribute]<=variable][t_data[Class] == target_variables[0]])
        den = len(t_data[t_data[attribute]<=variable])
        fraction = float(num)/(den)
        entropy = min(fraction, 1 - fraction)
        fraction2 = float(den)/total_rows
        entropy2 += fraction2*entropy
        
        num = len(t_data[t_data[attribute]>variable][t_data[Class] == target_variables[0]])
        den = len(t_data[t_data[attribute]>variable])
        fraction = float(num)/(den)
        entropy = min(fraction, 1 - fraction)
        fraction2 = float(den)/total_rows
        entropy2 += fraction2*entropy
#         ipdb.set_trace()
        
        if(split_node_entropy > abs(entropy2)):
            split_node_entropy = abs(entropy2)
            split_node = variable
            
    return split_node_entropy, split_node
#     Class = "left" 
#     class_labels = train_data[Class].unique() 
#     attribute_labels = train_data[attribute].unique()
    
#     if(len(attribute_labels) == 1):
#         return sys.float_info.max, attribute_labels[0]
    
#     class_labels.sort()
#     attribute_labels.sort()
#     combined = pd.DataFrame(train_data,columns=[attribute,'left']).sort_values(by=[attribute,'left'])
    
#     split_node_miss_class = None
#     split_node = None
#     np_array_length = len(combined)
#     total0 = len(combined[combined['left'] == 0])
#     total1 = np_array_length - total0
#     count0 = 0
#     i = 0
#     previous = None
#     for row in combined.iterrows():
#         if (i != 0 and previous != row[1][attribute]):
#             gini = 0
#             gini2 = 0
#             num = count0 
#             den = i
#             temp = num / (den)
#             gini = min(temp, (1 - temp))
            
#             temp2 = den/np_array_length
#             gini2 += temp2*gini
            
#             gini = 0
#             num = total0 - count0
#             den = np_array_length - i
#             temp = num / (den+eps)
#             gini = min(temp, (1 - temp))
            
#             temp2 = den/np_array_length
#             gini2 += temp2*gini
            
#             if(split_node_miss_class == None):
#                 split_node_miss_class = abs(gini2)
#                 split_node = previous
#             else:
#                 if(split_node_miss_class > abs(gini2)):
#                     split_node_miss_class = abs(gini2)
#                     split_node = previous
#         if (row[1]['left']==0):
#             count0 += 1
#         previous = row[1][attribute]
#         i += 1
        
#     gini = 0
#     gini2 = 0
#     num = count0
#     den = i
#     temp = num / (den)
#     gini = min(temp, (1 - temp))
    
#     temp2 = den/np_array_length
#     gini2 += temp2 * gini

#     gini = 0
#     num = total0 - count0 
#     den = np_array_length - i
#     temp = num / (den+eps)
#     gini = min(temp, (1 - temp))
    
#     temp2 = den/np_array_length
#     gini2 += temp2 * gini

#     if(split_node_miss_class == None):
#         split_node_miss_class = abs(gini2)
#         split_node = previous
#     else:
#         if(split_node_miss_class > abs(gini2)):
#             split_node_miss_class = abs(gini2)
#             split_node = previous
#     return split_node_miss_class, split_node

In [24]:
def entropy_on_discrete_attribute_split(t_data, attribute):
    """ Finds resulting entropy of dataset if it is split using attribute """
    miss_class_after_split = 0
    dependent_variable = "left"
    attribute_labels = t_data[attribute].unique()
    if(len(attribute_labels) == 1):
        return sys.float_info.max
    
    for attribute_label in attribute_labels:
        miss_class = 0
        numer = len(t_data[attribute][t_data[attribute] == attribute_label][t_data[dependent_variable] == 0])
        denom = len(t_data[attribute][t_data[attribute] == attribute_label])
        temp = float(numer) / (denom)
        miss_class = min(temp, (1 -  temp))
        temp2 = float(denom) / (len(t_data))
        miss_class_after_split += temp2 * miss_class
    return abs(miss_class_after_split)

In [25]:
def split_criteria(t_data):
    """ Finds the best attribute to split on """

    min_entropy = sys.float_info.max
    split_point = None
    min_entropy_attribute = None
    
    for key in discrete_attributes:
        entropy = entropy_on_discrete_attribute_split(t_data, key)
        if entropy < min_entropy:
            min_entropy = entropy
            min_entropy_attribute = key
        
    for key in real_attributes:
        entropy, entropy_val = entropy_on_real_attribute_split(t_data, key)
        if entropy < min_entropy:
            min_entropy = entropy
            split_point = entropy_val
            min_entropy_attribute = key
                      
    return min_entropy, min_entropy_attribute, split_point

In [47]:
def split_dataset_smaller(t_data, feature, split_point):
    return t_data[t_data[feature] <= split_point].reset_index(drop = True)

In [48]:
def split_dataset_greater(t_data, feature, split_point):
      return t_data[t_data[feature] > split_point].reset_index(drop = True)

In [49]:
def split_dataset(t_data, feature, label):
    """ This splits the dataset on given feature and all of its values """
    return t_data[t_data[feature] == label].reset_index(drop = True)

In [50]:
def most_probable(t_data):
    dependent_variable = "left"
    count_left = len(t_data[t_data[dependent_variable] == 0])
    count_right = len(t_data[t_data[dependent_variable] == 1])
    if count_left > count_right:
        return 0
    else:
        return 1

In [51]:
def Decision_tree(t_data):
    """ Builds tree recursively """
    Class = "left"
    D_tree = {}
    dependent_variable = "left"
#     ipdb.set_trace()

    target_variables = t_data[dependent_variable].unique()
    if len(target_variables) == 1:
        return target_variables[0]
#     ipdb.set_trace()
    min_entropy, root, split_point = split_criteria(t_data)
#     ipdb.set_trace()
    if min_entropy == sys.float_info.max:
        print (min_entropy)
        return most_probable(t_data)
    
     
    D_tree[root] = {}
    
    if root in (discrete_attributes):
        print (min_entropy, root)
        labels = t_data[root].unique()
        for label in labels:
            split_data = split_dataset(t_data, root, label)
            D_tree[root][label] = Decision_tree(split_data)
    else:
        print (min_entropy, root, split_point)
        split_data = split_dataset_smaller(t_data, root, split_point)
        D_tree[root][split_point] = Decision_tree(split_data)
            
        split_data = split_dataset_greater(t_data, root, split_point)
        D_tree[root][float(split_point) + 0.0001] = Decision_tree(split_data)

    return D_tree
    

In [52]:
def predict(inst, tree):
    for nodes in tree.keys():        
        value = inst[nodes]
        if nodes in discrete_attributes:
            if value in list((tree[nodes]).keys()):
                tree = tree[nodes][value]
            else:
                zeros = 0
                ones = 0
                for i in tree[nodes].keys():
                    if tree[nodes][i] == 0:
                        zeros += 1
                    elif tree[nodes][i] == 1:
                        ones += 1
                if zeros > ones:
                    return 0
                else:
                    return 1

        elif nodes in real_attributes:
            first_key = list(tree[nodes].keys())[0]
            if value <= first_key :
                tree = tree[nodes][first_key]
            else:
                second_key = list(tree[nodes].keys())[1]
                tree = tree[nodes][second_key]
        prediction = 0

        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;

    return prediction

In [53]:
def validate_tree(val_data, tree):
    predicted = []
    for index, row in val_data.iterrows():
        predicted.append(predict(row, tree))
    actual = val_data["left"].tolist()
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    
    for i in range(0, len(predicted)):
        if (predicted[i] == 0 and actual[i] == 0):
            true_neg += 1
        elif (predicted[i] == 0 and actual[i] == 1):
            false_neg += 1
        elif (predicted[i] == 1 and actual[i] == 0):
            false_pos += 1
        else:
            true_pos += 1
    return true_pos, true_neg, false_pos, false_neg 
    
        

In [54]:
def accuracy(val_data, tree):
    true_pos, true_neg, false_pos, false_neg = validate_tree(val_data, tree)
#     print (true_pos, true_neg, false_pos, false_neg)
    total_instances = true_neg + true_pos + false_neg + false_pos
    accuracy_estimate = (true_neg + true_pos) / (total_instances)
    precision_estimate = true_pos / (true_pos + false_pos)
    recall_estimate = (true_pos) / (true_pos + false_neg)
    f1_score = (1 / recall_estimate) + (1 / precision_estimate)
    f1_score = 2 / f1_score
    print ("Accuracy : ", accuracy_estimate)
    print ("Precision : ", precision_estimate)
    print ("Recall : ", recall_estimate)
    print ("F1_Score : ", f1_score)

In [55]:
tree = Decision_tree(train_data)
# accuracy(val_data, tree)

/home/kaushik/.local/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/kaushik/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.1759733036707453 satisfaction_level 0.11
0.13338071547026775 number_project 2
0.0994277539341917 satisfaction_level 0.46
0.04832347140039446 last_evaluation 0.57
0.027542372881355894 average_montly_hours 161
0.015184381778741887 last_evaluation 0.44
0.007683863885839721 satisfaction_level 0.31
0.0033185840707964263 average_montly_hours 125
0.003333333333333285 last_evaluation 0.56
0.0023952095808382847 average_montly_hours 153
0.0031055900621117546 average_montly_hours 128
0.0034305317324184728 satisfaction_level 0.32
0.003442340791738333 satisfaction_level 0.4
0.0036900369003689698 average_montly_hours 134
0.004694835680751126 last_evaluation 0.46
0.005405405405405373 average_montly_hours 143
0.009174311926605472 satisfaction_level 0.36
0.010309278350515427 satisfaction_level 0.39
0.014705882352941138 average_montly_hours 145
0.016949152542372854 average_montly_hours 146
0.018518518518518497 last_evaluation 0.52
0.026315789473684195 average_montly_hours 150
0.0 sales
0.0032258064516

0.08077246011754823 last_evaluation 0.37
0.08097643097643092 last_evaluation 0.57
0.011834319526627167 average_montly_hours 114
0.012237762237762184 average_montly_hours 184
0.016528925619834656 satisfaction_level 0.43
0.0 sales
0.0145530145530145 satisfaction_level 0.44
0.014799154334038 satisfaction_level 0.84
0.01652892561983466 satisfaction_level 0.74
0.01792114695340497 satisfaction_level 0.46
0.018587360594795495 average_montly_hours 163
0.027027027027026973 satisfaction_level 0.61
0.02702702702702699 satisfaction_level 0.47
0.028169014084507008 satisfaction_level 0.58
0.03448275862068961 last_evaluation 0.43
0.04255319148936166 average_montly_hours 121
0.044444444444444405 promotion_last_5years
0.045454545454545414 time_spend_company 2
0.060606060606060566 Work_accident
0.07692307692307689 last_evaluation 0.45
0.07999999999999997 satisfaction_level 0.5
0.0 sales
0.02702702702702699 average_montly_hours 130
0.028169014084507008 satisfaction_level 0.65
0.0 salary
0.008264462809917

0.0 average_montly_hours 161
0.015384615384615344 last_evaluation 0.81
0.016949152542372854 satisfaction_level 0.58
0.018518518518518483 last_evaluation 0.93
0.027027027027026994 satisfaction_level 0.73
0.0 average_montly_hours 163
0.008888888888888839 average_montly_hours 196
0.06666666666666662 satisfaction_level 0.51
0.07692307692307691 satisfaction_level 0.58
0.0 sales
0.00790960451977396 last_evaluation 0.98
0.008139534883720878 time_spend_company 2
0.012698412698412648 satisfaction_level 0.48
0.013114754098360604 satisfaction_level 0.61
0.027522935779816477 average_montly_hours 209
0.034883720930232516 Work_accident
0.027027027027026973 average_montly_hours 246
0.027027027027026994 satisfaction_level 0.58
0.03448275862068962 satisfaction_level 0.49
0.04545454545454543 last_evaluation 0.86
0.07692307692307691 salary
0.0 sales
0.027027027027027004 satisfaction_level 0.53
0.04166666666666663 satisfaction_level 0.54
0.04545454545454543 average_montly_hours 262
0.07692307692307691 sal

0.07999999999999997 salary
0.0 average_montly_hours 158
0.12724014336917566 last_evaluation 0.81
0.09375000000000004 average_montly_hours 219
0.0 satisfaction_level 0.83
0.08602150537634404 average_montly_hours 230
0.0 last_evaluation 0.8
0.0625 sales
0.029411764705882314 satisfaction_level 0.62
0.03448275862068962 satisfaction_level 0.69
0.04545454545454543 satisfaction_level 0.88
0.07692307692307691 satisfaction_level 0.77
0.0 salary
0.0 satisfaction_level 0.82
0.14285714285714282 Work_accident
0.16666666666666663 satisfaction_level 0.44
0.0 average_montly_hours 260
0.0 Work_accident
0.07359307359307357 time_spend_company 6
0.045977011494252915 satisfaction_level 0.7
0.14999999999999997 satisfaction_level 0.5
0.0 average_montly_hours 219
0.13333333333333328 average_montly_hours 240
0.09090909090909088 last_evaluation 0.91
0.0 sales
0.02168674698795182 number_project 3
0.014742014742014755 number_project 5
0.012376237623762382 satisfaction_level 0.92
0.012406947890818809 average_montl

In [56]:
tree


{'satisfaction_level': {0.11: 1,
  0.1101: {'number_project': {2: {'satisfaction_level': {0.46: {'last_evaluation': {0.57: {'average_montly_hours': {161: {'last_evaluation': {0.44: 0,
            0.4401: {'satisfaction_level': {0.31: 0,
              0.3101: {'average_montly_hours': {125: 0,
                125.0001: {'last_evaluation': {0.56: {'average_montly_hours': {153: {'average_montly_hours': {128: 1,
                      128.0001: {'satisfaction_level': {0.32: 1,
                        0.3201: {'satisfaction_level': {0.4: {'average_montly_hours': {134: 1,
                            134.0001: {'last_evaluation': {0.46: 1,
                              0.4601: {'average_montly_hours': {143: 1,
                                143.0001: {'satisfaction_level': {0.36: 1,
                                  0.3601: {'satisfaction_level': {0.39: {'average_montly_hours': {145: 1,
                                      145.0001: {'average_montly_hours': {146: 1,
                          

In [57]:
accuracy(val_data, tree)

Accuracy :  0.2459964412811388
Precision :  0.23648029330889092
Recall :  0.9467889908256881
F1_Score :  0.37843784378437845
